In [ ]:
# Importing necessary libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
apps = pd.read_csv("../input/google-playstore-data/googleplaystore.csv")
apps.head(5)

In [ ]:
apps.shape

In [ ]:
apps.describe()

In [ ]:
from dateutil import parser
s= 'April 25, 2020'
p = parser.parse(s)
p = p.strftime("%d/%m/%Y")
print(p)
print(type(p))

In [ ]:
def Mapper(x):
  try:
    date = parser.parse(x) # datetime.datetime(2020, 4, 25, 0, 0)
    date_string = date.strftime("%d/%m/%Y")
    return date_string
  except:
    return np.nan

apps['Last Updated'] = apps['Last Updated'].map(Mapper)

In [ ]:
apps.isnull().sum()

In [ ]:
apps = apps.dropna(how = 'any')

apps.shape

In [ ]:
apps['Last Updated'] = pd.to_datetime(apps['Last Updated']) # format='%m-%d-%Y')

In [ ]:
apps.head(2)

In [ ]:
size_list = apps['Size'].to_list()
sizes = []
def extractingSize(item):
  count = 0
  try:
    s = item[:len(item) - 1]
    sizes.append(float(s))
    return float(s)
  except:
    count = count + 1
    pass

print(size_list[:6])
apps['Size'] = apps['Size'].map(extractingSize)

In [ ]:
9630 - len(sizes)

In [ ]:
apps.head()

In [ ]:
apps["Size"].astype('float64')

apps['Size'].isnull().sum()

In [ ]:
apps = apps.dropna(how = 'any')

In [ ]:
s = '4.0.3'.split(".")
print(s[0] + "." + s[1] + s[2])

In [ ]:
print(len(apps['Type'].value_counts()))
apps['Type'].value_counts()

In [ ]:
print(len(apps['Content Rating'].value_counts()))
apps['Content Rating'].value_counts()

In [ ]:
print(len(apps['Genres'].value_counts()))
apps['Genres'].value_counts()

In [ ]:
print(len(apps['Category'].value_counts()))
apps['Category'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(apps['Category'])
apps['Category'] = le.transform(apps['Category'])

In [ ]:
print(le.classes_) # ART_AND_DESIGN is given 1

In [ ]:
le.fit(apps['Content Rating'])
apps['Content Rating'] = le.transform(apps['Content Rating'])

print(le.classes_)

In [ ]:
# [0 , 1, 2, 3, 4, 5]

In [ ]:
def ref2(x):
  if x == 'Paid':
    return 1
  else:
    return 0

apps['Type'] = apps['Type'].map(ref2)

In [ ]:
apps = apps.drop([ 'App','Genres', 'Current Ver'], axis = 1)

apps.head()

In [ ]:
install_list = apps['Installs'].to_list()
installs = []
def extractingInstalls(x):
  for item in x:
    if not item.startswith("Varies"):
      s = str(item[:len(item) - 1])
      s = s.replace(",", "")
      installs.append(float(s))
    else:
      installs.append(1000.0)

extractingInstalls(install_list)
print(install_list[:6])
apps['Installs'] = installs

In [ ]:
def ref3(x):
  return float(x)

apps['Reviews'] = apps["Reviews"].map(ref3)

In [ ]:
import datetime as dt

apps['year'] = apps['Last Updated'].dt.year
apps['month'] = apps['Last Updated'].dt.month
apps['day'] = apps['Last Updated'].dt.day

In [ ]:
ver_list = apps['Android Ver'].to_list()
ver = []
def extractingAndroid(x):
  for item in x:
    if not item.startswith("Varies"):
      s = str(item[:3])
      ver.append(float(s))
    else:
      ver.append(4.4)

extractingAndroid(ver_list)
print(ver_list[:6])
apps['Android Ver'] = ver

In [ ]:
price_list = apps['Price'].to_list()
prices = []
def extractingPrice(x):
  for item in x:
    if item != 0:
      s = item.replace("$", "")
      prices.append(float(s))
    else:
      prices.append(0.0)

extractingPrice(price_list)
print(price_list[:6])
apps['Price'] = prices

In [ ]:
apps.isnull().sum()

In [ ]:
apps.head(25)

In [ ]:
apps.isnull().sum()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(apps.corr(),annot=True,cmap='viridis',linewidths=.5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
y = apps['Rating']
X = apps.drop(["Rating", "Last Updated", "Reviews", "Android Ver"], axis = 1)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 5)
pca_fit = pca.fit_transform(X[:1000])

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(pca_fit, y[:1000], test_size=0.2)

In [ ]:
random_model = RandomForestRegressor(n_estimators=200, n_jobs = -1, random_state = 42)

In [ ]:
#Fit
random_model.fit(Xtrain, ytrain)

y_pred = random_model.predict(Xtest)

#Checking the accuracy
random_model_accuracy = round(random_model.score(Xtrain, ytrain)*100,2)
print(round(random_model_accuracy, 2), '%')

In [ ]:
random_model_accuracy1 = round(random_model.score(Xtest, ytest)*100,2)
print(round(random_model_accuracy1, 2), '%')

In [ ]:
# Save the trained model as a pickle string.
import pickle 

saved_model = pickle.dump(random_model, open('Apps.pickle','wb'))

saved_pca = pickle.dump(pca, open('AppsPCA.pickle','wb'))